# BreakItDown Tasks

### 1. Mark video as watched
- If ID exist
- query result skip over id and check again if ID exists
- repeat until not 

### 2. Revised percentage calculation (relative instead of 100%)
Will cause a tag to be dominant tho (weight 1000 while others still 10). Mitigated with intentional interruptions (#4). 

### 3. User class schema
- self
- interest_weights (dict)
- watched_videos (dict)

### 4. Intentional Interruptions
Handle dominations
- 50% -> Suggest relevant interests 
- 75% -> "Have you been enjoying {tag name} so far? "yes". "recommend more". 

### 5. Modify whisper prompt to return most relevant tag first then extract that for on_user_swipe()
in [github](https://github.com/edgurinc/edgur/blob/main/backend/accounts/videoprocessors.py)
- To help us later in handling which tag to add weights to given videos have multiple tags. 

### 6. Decay other interests whenever watching a video
except current video tag 

### 7. Handle irrelevant tags
Acknowledge that it will be filled soon, such as "food" category

### 8. "Cache" next 4 videos
Run uniformDist+reco only on 4th video 

### 9. Min selection Max selection for categories
Like tiktok's

### 10. Show this to me again (#SpacedRepetition)
Remove from watched dictionary

___
# User Interest Algorithm


In [37]:
from pprint import pprint
import random

# Constants

new_video_weight = 3
starting_weight = 3

user_min_interest = 4
user_max_interest = 8

queued_videos_count = 3
# Number of vids to queue before running query again (to save costs)
# AKA n_results = 3.

# User Actions
Like = 0.1
Share = 0.2
Watch = 0.1  # more than 50% of total duration
Loop = 0.1

natural_decay_rate = 0.1

# Dictionaries - remove
watched_videos = {}


# Simulation Only (replace user input)
chosen_tag = "Microwave"
user1_interests = ["DevOps", "Tests (SAT)", "Machine Learning", "Cats", "Cooking"]


In [38]:
def create_user_interest_dict(user_interest_list):
    user_dict_interest = {interest: starting_weight for interest in user_interest_list}
    user_dict_interest["Random"] = starting_weight
    return user_dict_interest

In [39]:
# HELPER FUNCTIONS for update_weight

# User actions

def observe_action_taken(video_tag, liked=False, shared=False, watched=False, loop_count=0):
    """
    This function calculates the total points based on the user's actions.

    Parameters:
    video_tag (str): The tag of the video.
    liked (bool): Whether the user liked the video.
    shared (bool): Whether the user shared the video.
    watched (bool): Whether the user watched the video.
    loop_count (int): The number of times the user looped the video.

    Returns:
    float: The total points calculated based on the user's actions.
    """
    # Actions and their corresponding points to add
    Actions = {
        "Like": Like,
        "Share": Share,
        "Watch": Watch,  # more than 50% of total duration
        # todo: experiment with increasing watch value
        "Loop": Loop,
    }

    total_action_points = 0

    if liked:
        total_action_points += Actions["Like"]
    if shared:
        total_action_points += Actions["Share"]
    if watched:
        total_action_points += Actions["Watch"]
    total_action_points += loop_count * Actions["Loop"]

    print(f"\nUpdating weights for video tag: {video_tag}")
    print(f"Total points to add: {total_action_points}")

    return total_action_points

def handle_update_weight(user_dict_interest, video_tag, total_action_points):

    # Increase the weight of the interest based on the action points
    user_dict_interest[video_tag] += total_action_points

    for interest in user_dict_interest:
        if interest != "Random" and interest != video_tag:
            user_dict_interest[interest] -= natural_decay_rate

    return user_dict_interest

def handle_add_new_tag(user_dict_interest, video_tag):
    # Add new interest if it doesn't exist and give initial points
    user_dict_interest[video_tag] = starting_weight

    # Decrease the weight of all other interests by the natural decay rate
    for interest in user_dict_interest:
        if interest != "Random" and interest != video_tag:
            user_dict_interest[interest] -= natural_decay_rate

    return user_dict_interest

In [40]:
def watch_video_and_update_weight(
    user_dict_interest,
    video_tag,
    liked=False,
    shared=False,
    watched=False,
    loop_count=0,
):

    total_points = observe_action_taken(video_tag, liked, shared, watched, loop_count)

    if video_tag in user_dict_interest:
        handle_update_weight(user_dict_interest, video_tag, total_points)
    else:
        handle_add_new_tag(user_dict_interest, video_tag)

    # Sort the interests ("key=item[1]") by weight in descending order and keep only the top 10
    user_dict_interest = dict(
        sorted(user_dict_interest.items(), key=lambda item: item[1], reverse=True)[:10]
    )


    return user_dict_interest

In [55]:
def get_random_tag_for_recommendation(user_dict_interest):

    total_score = sum(value for value in user_dict_interest.values())
    random_value = round(random.uniform(0, total_score), 0)

    print("_" * 50)
    print(f"\nRandom value: {random_value}\n")

    cumulative_probability = 0.0
    partitions = []

    for interest, probability in user_dict_interest.items():
        # Save the starting point of the current tag's range (in probability)
        previous_cumulative_probability = cumulative_probability

        # Add the tag's probability to the cumulative total to get the end point of the tag's range
        cumulative_probability += probability

        partitions.append(
            (interest, previous_cumulative_probability, cumulative_probability)
        )

        pprint(
            f"Checking interest: {interest}, cumulative range: {previous_cumulative_probability} - {cumulative_probability}"
        )

        # If the random value is less than or equal to the cumulative probability up to the current interest,
        # select the current interest and stop looking at the rest of the interests.
        if random_value <= cumulative_probability:
            print(f"\nSelected interest: {interest}\n")
            break

    if interest == "Random":
        interest = random.choice(list(user_dict_interest.keys()))
        print(f"Random tag selected: {interest}")
        print("MODIFY THIS LATER TO POINT TO CSV COLUMN")
        return interest

    return interest

In [57]:
# Sample usage
user_interests = create_user_interest_dict(user1_interests)
print("Initial user-chosen interests\n")
pprint(f"{user_interests}\n")

total_action_points = observe_action_taken("Cooking", True, False, True, 0)

watch_video_and_update_weight(user_interests, "Shiba", True, False, True, 0)

get_random_tag_for_recommendation(user_interests)

pprint(user_interests)

Initial user-chosen interests

("{'DevOps': 3, 'Tests (SAT)': 3, 'Machine Learning': 3, 'Cats': 3, 'Cooking': "
 "3, 'Random': 3}\n")

Updating weights for video tag: Cooking
Total points to add: 0.2

Updating weights for video tag: Shiba
Total points to add: 0.2
__________________________________________________

Random value: 16.0

'Checking interest: DevOps, cumulative range: 0.0 - 2.9'
'Checking interest: Tests (SAT), cumulative range: 2.9 - 5.8'
'Checking interest: Machine Learning, cumulative range: 5.8 - 8.7'
'Checking interest: Cats, cumulative range: 8.7 - 11.6'
'Checking interest: Cooking, cumulative range: 11.6 - 14.5'
'Checking interest: Random, cumulative range: 14.5 - 17.5'

Selected interest: Random

Random tag selected: Random
MODIFY THIS LATER TO POINT TO CSV COLUMN
{'Cats': 2.9,
 'Cooking': 2.9,
 'DevOps': 2.9,
 'Machine Learning': 2.9,
 'Random': 3,
 'Shiba': 3,
 'Tests (SAT)': 2.9}


___
# Ranking Algorithm


In [43]:
import pandas as pd
from pprint import pprint
import chromadb
from chromadb.utils import embedding_functions

chroma_client = chromadb.PersistentClient(path="db")

# Use OpenAI as the embedding model (word to vector)
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key="sk-",
                model_name="text-embedding-3-small"
            )

# Set the URL and initialize ChromaDB client
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vSAE2tBAnAdXsxk9a9YClFN7MSEVhzEmJD01ewwtooMLxL-Ilod26EbdD8sZeZk0ybiqD-jqT-9RZbn/pub?gid=497214901&single=true&output=csv"
df = pd.read_csv(url)

# Create a vectorstore database and use Cosine Similarity for semantic search within the DB
collection = chroma_client.get_or_create_collection(
    name="video_collection",
    embedding_function=openai_ef,
    metadata={"hnsw:space": "cosine"},
)

watched_videos = {}

In [44]:
# Function for adding new rows to the collection

# Convert the 'tags' column to a list of strings (grouped according to its respective row)
documents = df["tags"].apply(lambda x: x.split(",")).tolist()
documents_str = [", ".join(doc) for doc in documents]

# Add the documents to the collection
# todo: handle UUIDs
ids = [str(i + 1) for i in range(len(documents_str))]

# Add new rows to the collection
collection.upsert(documents=documents_str, ids=ids)

In [45]:
# Query the collection for the top 3 most similar videos